<a href="https://colab.research.google.com/github/Farama-Foundation/MO-Gymnasium/blob/main/mo_gymnasium_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: install and import the libraries

In [1]:
!pip install mo-gymnasium

In [1]:
import sys
!{sys.executable} -m pip install moviepy

In [2]:
!pip install gymnasium
import gymnasium as gym
import mo_gymnasium as mo_gym

## Step 2: create an environment

In [4]:
gym.envs.register(
    id="mo-mountaincar-2Obj",
    entry_point="mo_gymnasium.envs.mountain_car_2obj.mountain_car_2obj:MOMountainCar",
    max_episode_steps=200,
)
env = mo_gym.make("mo-mountaincar-2Obj", render_mode="rgb_array") #Why does it not detect the new 

Box(-1.0, [-1.  0.], (2,), float32)


/Users/Pranav/miniconda3/envs/dissertation/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment mo-mountaincar-2Obj already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/Pranav/miniconda3/envs/dissertation/lib/python3.8/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## Step 3: extract environment information

In [7]:
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [8]:
env.action_space

Discrete(3)

In [9]:
env.reward_space

/Users/Pranav/miniconda3/envs/dissertation/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  env_spec.additional_wrappers += (wrapper_spec,)


Box(-1.0, [-1.  0.  0.], (3,), float32)

## Step 4: use Gymnasium features in MO-Gymnasium

In [5]:
from gymnasium.wrappers.record_video import RecordVideo

In [6]:
env = RecordVideo(env, "videos/demo", episode_trigger=lambda e: True)

/Users/Pranav/miniconda3/envs/dissertation/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:87: UserWarning: WARN: Overwriting existing videos at /Users/Pranav/Desktop/Year4/Dissertation/Code & Papers/Repos/MO-Gymnasium/videos/demo folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Step 5: go through the environment with a random agent

In [7]:
#Kernel works with random agent
env.reset()
done = False

while not done:
    obs, vec_reward, terminated, truncated, info = env.step(env.action_space.sample())
    done = terminated or truncated

Moviepy - Building video /Users/Pranav/Desktop/Year4/Dissertation/Code & Papers/Repos/MO-Gymnasium/videos/demo/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/Pranav/Desktop/Year4/Dissertation/Code & Papers/Repos/MO-Gymnasium/videos/demo/rl-video-episode-0.mp4



TypeError: must be real number, not NoneType

## Step 6: scalarize env and run it with stable-baselines3



In [8]:
!pip install stable-baselines3 --upgrade

In [12]:
import stable_baselines3 as sb3
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np
import mo_gymnasium as mo_gym

In [10]:
# Linear scalarizes the environment
env = mo_gym.LinearReward(mo_gym.make("mo-mountaincar-v0"), weight=np.array([0.9, 0.1, 0.0]))

# Run DQN agent!
agent = sb3.DQN("MlpPolicy", env)
agent.learn(10000)

/Users/Pranav/miniconda3/envs/dissertation/lib/python3.8/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [13]:
mean_reward, std_reward = evaluate_policy(agent, agent.get_env(), n_eval_episodes=10) #Originally 10
print(f"Mean reward: {mean_reward} and Standard Deviation: {std_reward}")

Mean reward: -180.0 and Standard Deviation: 0.0


In [14]:
# Linear scalarizes the environment
env = mo_gym.LinearReward(mo_gym.make("mo-mountaincar-v0", render_mode="rgb_array"), weight=np.array([1, 0, 0])) #0.9, 0.1, 0.0
#Can't solve with the scalarization/DQN [Not sure which one]
print("Action space:", env.action_space)
# for action_index in range(env.action_space.n):
#         print(f"Action index {action_index} corresponds to action: {env.unwrapped.get_action_meanings()[action_index]}")


# Medium Post:
# agent = sb3.DQN(policy="MlpPolicy",
#     env=env,
#     seed=0,
#     batch_size=256,
#     learning_rate=7.77e-05,
#     buffer_size=int(1e5),  # Replay buffer size
#     learning_starts=1000,  # Number of steps before starting training
#     train_freq=4,          # Number of actions between updates
#     gradient_steps=1,      # Number of optimization steps per batch
#     target_update_interval=1000,  # Frequency (in steps) to update the target network
#     exploration_fraction=0.1,     # Fraction of the total training steps for exploration
#     exploration_final_eps=0.02,   # Final epsilon value for exploration
#     gamma=0.9999,
#     max_grad_norm=5,
#     policy_kwargs=None,
#     verbose=1)

agent = sb3.DQN(policy="MlpPolicy",
    env=env,
    seed=0,
    learning_rate=0.0001,
    buffer_size=int(2e5),  # Replay buffer size
    learning_starts=1000,  # Number of steps before starting training
    train_freq=4,          # Number of actions between updates
    gradient_steps=1,      # Number of optimization steps per batch
    target_update_interval=1000,  # Frequency (in steps) to update the target network
    exploration_fraction=1,     # Fraction of the total training steps for exploration
    exploration_final_eps=0,   # Final epsilon value for exploration
    gamma=0.9,
    max_grad_norm=5,
    policy_kwargs=None,
    verbose=0)

#Result keeps coming as -180 without any standard deviation [Issue with the training]
agent.learn(total_timesteps=200000,progress_bar=True)  #Number of episodes?

#Can save each model I learn and see which one is the best
#timesteps is 200000
mean_reward, std_reward = evaluate_policy(agent, agent.get_env(), n_eval_episodes=10) #Originally 10
print(f"Mean reward: {mean_reward} and Standard Deviation: {std_reward}")

Output()

Action space: Discrete(3)


Mean reward: -200.0 and Standard Deviation: 0.0
